# Motivating Recursion: When Loops Get Tedious

Recursion is an extremely powerful feature of programming languages, and it can appear intimidating, but it's worth learning.  [One popular blog post from back in the day](https://www.joelonsoftware.com/2005/12/29/the-perils-of-javaschools-2/) even calls out recursion as one of the defining challenges of a computer-science education, such that companies should try to check for understanding when interviewing programmers.

Let's try to demystify recursion with a recipe for thinking about how to apply it.  First, some examples will help us see why recursive programs can be so much more compact and understandable.  It's not hard to sum up the numbers in a list, using a loop.

In [1]:
def list_sum(L):
    """Return the sum of the elements in a list."""

    total = 0
    for x in L:
        total += x
    return total

In [2]:
assert list_sum([]) == 0
assert list_sum([1]) == 1
assert list_sum([1, 2, 3]) == 6
assert list_sum([1, -1, 2, -2, 100, -100, 6]) == 6

So far so good.  That problem *is* a good match for loops.  What about if we want to sum all the numbers in lists of lists of lists ... of numbers, for an arbitrary depth?  (See the test cases below for help understanding what we are trying to do.)  Now we have to work harder, essentially maintaining a classic data structure called a *stack*, which is essentially a list where we periodically add elements to the front or remove elements from the front -- we never "skip into" the list and change its middle or end.

In [3]:
def sum_all(L):
    """Return the sum of all elements in a list with arbitrary nesting."""

    total = 0
    while len(L) > 0:
        if isinstance(L[0], list):
            # Oh, the first value is a list? 
            # Rather then adding it to the total, let's smoosh it together with the input.
            L = L[0] + L[1:]
        else:
            # The first value must be something we add directly into our total.
            total += L[0]
            del L[0] # Remove first list element.
    return total

In [4]:
assert sum_all([1, 2, 3]) == 6
assert sum_all([1, [2, 3], 4]) == 10
assert sum_all([1, 2, [3, [4, [[[5]]]], 6, [7, 8]], 9]) == 45

That was mildly exhausting and twisty, but it can get worse!  What if we want a *weighted* sum, where each number adds to the total in proportion to how deep it is in the nesting?  We now use our stack in a fancier way, recording when a depth multiplier needs to be decreased.

In [5]:
def weighted_sum_all(L):
    """Return the sum of all elements in a list with arbitrary nesting, with a catch:
    Every addition to the sum should be *multiplied* by the depth where it appears.
    The first level of list has depth 1, then the next level has depth 2, etc."""

    total = 0
    multiplier = 1
    while len(L) > 0:
        if isinstance(L[0], list):
            # Oh, the first value is a list?  We got more nested, so bump multiplier.
            multiplier += 1
            # Rather then adding the nested list to the total, let's smoosh it together with the input.
            # We also leave a reminder of when to decrease the multiplier again.
            L = L[0] + ['decrease'] + L[1:]
        elif L[0] == 'decrease':
            multiplier -= 1
            del L[0]
        else:
            # The first value must be something we add directly into our total.
            total += multiplier * L[0]
            del L[0]
    return total

In [6]:
assert weighted_sum_all([1]) == 1
assert weighted_sum_all([[1]]) == 2
assert weighted_sum_all([[[1]]]) == 3
assert weighted_sum_all([[1], 3]) == 5

## Let's Try That Again with Recursion

Recursion doesn't *always* make programs nicer.  For instance, it's debatable whether this recursive version of our first example has gotten more readable.

In [7]:
def list_sum(L):
    """Return the sum of the elements in a list."""

    if len(L) == 0:
        return 0
    else:
        return L[0] + list_sum(L[1:])

In [8]:
assert list_sum([]) == 0
assert list_sum([1]) == 1
assert list_sum([1, 2, 3]) == 6
assert list_sum([1, -1, 2, -2, 100, -100, 6]) == 6

Nonetheless, let's talk through a general recipe that can be followed to generate a definition like that one.  Fundamentally, recursion is about *defining a thing in terms of itself*, which sounds like cheating at first!  For instance, if we define a function that always calls itself with the same arguments, we run forever and never get an answer.  By following some informal rules, we can avoid "cheating" and get useful self-referential (recursive) code.

Here is the three-step recipe to write a recursive function, assuming you have already decided on what arguments the function takes and what it is supposed to *do*.

 1. *Divide the input space* into at least two regions.  In our list sum example, we split into empty lists and nonempty lists.
 2. *Identify some regions as **base cases** and solve them immediately* (without recursion).  The base case of our last example was the empty-list case, where the sum is obviously zero.
 3. *Solve the remaining regions as **recursive cases**,* where each makes one or more calls to the same function *but where the arguments have gotten smaller*.  In our sum example, we call `list_sum` recursively but on a *shorter* list, which counts as "getting smaller."

We can help understand recursion by instrumenting a function to print its arguments, along with an indicator of how deeply nested each call is.

In [9]:
def list_sum(L, indent=0):
    """Return the sum of the elements in a list."""

    print(indent*'.', L)
    if len(L) == 0:
        return 0
    else:
        return L[0] + list_sum(L[1:], indent+1)

In [14]:
assert list_sum([]) == 0
assert list_sum([1]) == 1
assert list_sum([1, 2, 3]) == 6
assert list_sum([1, -1, 2, -2, 100, -100, 6]) == 6

 []
 [1]
. []
 [1, 2, 3]
. [2, 3]
.. [3]
... []
 [1, -1, 2, -2, 100, -100, 6]
. [-1, 2, -2, 100, -100, 6]
.. [2, -2, 100, -100, 6]
... [-2, 100, -100, 6]
.... [100, -100, 6]
..... [-100, 6]
...... [6]
....... []


Now let's revisit summing up nested lists.  Recursion gives us a nicer version where we don't need to maintain a stack ourselves.

In [19]:
def sum_all(L):
    """Return the sum of all elements in a list with arbitrary nesting."""

    if len(L) == 0:
        return 0
    elif isinstance(L[0], list):
        # Oh, the first value is a list? 
        # We'll make *two* recursive calls for this case.
        return sum_all(L[0]) + sum_all(L[1:])
    else:
        # The first value must be something we add directly into our total.
        return L[0] + sum_all(L[1:])

In [22]:
assert sum_all([1, 2, 3]) == 6
assert sum_all([1, [2, 3], 4]) == 10
assert sum_all([1, 2, [3, [4, [[[5]]]], 6, [7, 8]], 9]) == 45

Actually, behind the scenes, Python maintains a *call stack* for us, so we don't need to maintain it ourselves!  It is common to diagram nested function calls as *frames* that pile up on top of each other, recording function names and arguments.  We can also add printing instrumentation to make Python show us a sense of how nesting works.

In [107]:
def sum_all(L, indent=0):
    """Return the sum of all elements in a list with arbitrary nesting."""

    print(indent*'.', L)
    
    if len(L) == 0:
        return 0
    elif isinstance(L[0], list):
        # Oh, the first value is a list? 
        # We'll make *two* recursive calls for this case.
        return sum_all(L[0], indent+1) + sum_all(L[1:], indent+1)
    else:
        # The first value must be something we add directly into our total.
        return L[0] + sum_all(L[1:], indent+1)

In [108]:
assert sum_all([1, 2, 3]) == 6
assert sum_all([1, [2, 3], 4]) == 10
assert sum_all([1, 2, [3, [4, [[[5]]]], 6, [7, 8]], 9]) == 45

 [1, 2, 3]
. [2, 3]
.. [3]
... []
 [1, [2, 3], 4]
. [[2, 3], 4]
.. [2, 3]
... [3]
.... []
.. [4]
... []
 [1, 2, [3, [4, [[[5]]]], 6, [7, 8]], 9]
. [2, [3, [4, [[[5]]]], 6, [7, 8]], 9]
.. [[3, [4, [[[5]]]], 6, [7, 8]], 9]
... [3, [4, [[[5]]]], 6, [7, 8]]
.... [[4, [[[5]]]], 6, [7, 8]]
..... [4, [[[5]]]]
...... [[[[5]]]]
....... [[[5]]]
........ [[5]]
......... [5]
.......... []
......... []
........ []
....... []
..... [6, [7, 8]]
...... [[7, 8]]
....... [7, 8]
........ [8]
......... []
....... []
... [9]
.... []


However, a word of caution: experts don't generally think in terms of call stacks and traces of function calls!  Instead, they apply the three-step recipe and trust that the results will therefore be good.  How did the recipe generate our last example?

 1. *Divide the input space*: empty lists, lists with lists as first elements, lists with other first elements
 2. *Base cases*: empty lists
 3. *Recursive cases*: solved with calls whose argument lists are always shorter than the inputs

Let's take a shot at weighted sums of nested lists.  Here we follow the same input-space division, and we add an extra argument to track the weight of the top-level list.  By passing different weights in different recursive calls, we get Python to keep track of a detail that we handled manually before with special `'decrease'` markers.

In [111]:
def weighted_sum_all(L, weight=1):
    """Return the sum of all elements in a list with arbitrary nesting, with a catch:
    Every addition to the sum should be *multiplied* by the depth where it appears.
    The first level of list has depth weight, then the next level has depth weigh+1, etc."""

    if len(L) == 0:
        return 0
    elif isinstance(L[0], list):
        return weighted_sum_all(L[0], weight+1) + weighted_sum_all(L[1:], weight)
    else:
        return weight * L[0] + weighted_sum_all(L[1:], weight)

In [113]:
assert weighted_sum_all([1]) == 1
assert weighted_sum_all([[1]]) == 2
assert weighted_sum_all([[[1]]]) == 3
assert weighted_sum_all([[1], 3]) == 5

# Making Change

Let's keep flexing our recursion muscles by applying the recipe in different variations.  Consider the convenience-store cashier's standard problem of trying to give the customer some change, from a supply of coins that are on-hand.  In more abstract terms, given a list of numbers (standing for the coins on-hand, therefore possibly including duplicates), we want to find a subset of them that add up to a particular target.

 1. *Divide the input space*: zero amount, empty coin list, any other case
 2. *Base cases*: the first two
 3. *Recursive cases*: solved by calling self where the coins list has shrunk

In [15]:
def make_change(amount, coins):
    """Let coins be a list of positive integer values for coins that are available for making change.
    Try to find a way to use just those coins to add up to a total of amount.
    If found, return a list of coins.  Otherwise, return None.
    """
    if amount == 0:
        # No coins, but also no more change left to make!  Easy solution.
        return []
    elif len(coins) == 0:
        # We have some change left to make, but no coins are available.
        return None
    else:
        # At least one coin remains.  Consider the cases we use it or don't use it.
        do_not_use_it = make_change(amount, coins[1:])
        if do_not_use_it != None:
            # Any solution that skips this coin is also legal if this coin was allowed, too.
            return do_not_use_it
        else:
            use_it = make_change(amount - coins[0], coins[1:])
            if use_it != None:
                # Any solution that skips this coin can be converted back to a global solution by adding the coin.
                # Note that we decreased amount in the recursive call, to ensure this trick works.
                return [coins[0]] + use_it
            else:
                return None

In [16]:
assert make_change(0, [1, 5, 10]) == []
assert make_change(15, [1, 5, 10]) == [5, 10]
assert make_change(11, [1, 5, 10]) == [1, 10]
assert make_change(13, [1, 5, 10]) == None

Let's not get complacent.  If we're not careful in applying the recursion recipe, we can easily get functions that run forever.  Consider the case where we allow any coin in the list to be used any number of times.

In [116]:
def make_change(amount, coins):
    """Let coins be a list of positive integer values for coins that are available for making change.
    Try to find a way to use just those coins to add up to a total of amount.
    If found, return a list of coins.  Otherwise, return None.
    """
    if amount == 0:
        # No coins, but also no more change left to make!  Easy solution.
        return []
    # DANGER ZONE: What if we forgot the following base case?
    elif amount < 0:
        # No way to make negative change!
        # If we skip this case, the recursion could continue forever (or at least for a very long time).
        return None
    elif len(coins) == 0:
        # We have some change left to make, but no coins are available.
        return None
    else:
        # At least one coin remains.  Consider the cases we use it or don't use it.
        do_not_use_it = make_change(amount, coins[1:])
        if do_not_use_it != None:
            # Any solution that skips this coin is also legal if this coin was allowed, too.
            return do_not_use_it
        else:
            use_it = make_change(amount - coins[0], coins)
            if use_it != None:
                # Any solution that skips this coin can be converted back to a global solution by adding the coin.
                # Note that we decreased amount in the recursive call, to ensure this trick works.
                return [coins[0]] + use_it
            else:
                return None

Notice that the second recursive call keeps the coin list the same.  We run into trouble with the recipe.

 1. *Divide the input space*: zero amount, empty coin list, any other case
 2. *Base cases*: the first two
 3. *Recursive cases*: solved by calling self where the coins list has shrunk... or, no, it hasn't (sometimes)!

We could try to salvage the argument.  For any given call, consider the sum of the amount and the number of coins.  This sum does actually decrease on each recursive call!  But the trouble is that *it can become negative*.  When we convince ourselves of termination by assigning sizes to arguments and arguing that sizes decrease on recursion, sizes must always be enforced as *nonnegative*, so that zero serves as a safe boundary on recursion depth.  (In theory, we would also be OK if we had, say, -365 as an enforced lower bound of the measure, but it's traditional to think in terms of zero.)

Concretely, what went wrong when we applied the recipe incorrectly?  We got stuck in an infinite digression of making the second recursive call, decreasing amount forever without changing coins.  Note that adding the extra base case for negative amount fixes the problem, and hence the measure proposed in the last paragraph *does* work to argue for termination.

In [118]:
print(make_change(0, [1, 5, 10]))
print(make_change(15, [1, 5, 10]))
print(make_change(11, [1, 5, 10]))
print(make_change(13, [1, 5, 10]))
print(make_change(7, [5, 10, 25]))

[]
[5, 10]
[1, 10]
[1, 1, 1, 10]
None


# Recursion and Trees

Let's play with nested data structures a bit more.

An abstract syntax tree (AST) is a convenient way of representing an arbitrary expression involving numbers and arithmetic operators.  Here's a description of an AST in Backus-Naur Form (BNF), a famous way of describing computer languages:

`ast :== integer | string | (operator, ast, ast)
operator :== "+" | "*" | …`

Here are the rules for determining the value of an AST, given an *environment* (a dictionary mapping variable names to values):

* If AST is an integer, return its value.
* If AST is a string, treat as a variable name, returning the variable's value.
* Otherwise, return the result of performing the operation on the values of the operands.

In [121]:
def eval_operator(operator, operand1, operand2):
    """Return the result of applying the given operator to the two numeric operands."""

    if operator == '+':
        return operand1 + operand2
    elif operator == '-':
        return operand1 - operand2
    elif operator == '*':
        return operand1 * operand2
    elif operator == '/':
        return operand1 / operand2
    else:
        raise ValueError

def eval_ast(ast, env={}):
    """Return the value of the AST in the given environment (dictionary mapping variables to values)."""

    if isinstance(ast, tuple):
        # If it's a tuple, it represents an operator.  Unpack the pieces and recurse.
        operator, operand1, operand2 = ast
        return eval_operator(operator, eval_ast(operand1, env), eval_ast(operand2, env))
    elif isinstance(ast, str):
        # It's a variable, represented as the variable name (string)?  Just look up in the environment.
        return env[ast]
    else:
        # It's a constant?  Then the AST is its own value.
        return ast

In [122]:
assert eval_ast(3) == 3
assert eval_ast('pi', {'pi': 3.141592653}) == 3.141592653
assert eval_ast(('*', 2, 21)) == 42
    
# evaluate 3 + 47*5/pi - 13
ast = ('-',
       ('+',
          ('/',
           ('*', 47, 5),
           'pi'),
          3),
       13)
assert eval_ast(ast, {'pi': 3.141592653}) == 64.80282326723406

Notice that our same old recipe can guide the design of this method.

 1. *Divide the input space*: operators, variables, constants
 2. *Base cases*: variables and constants
 3. *Recursive cases*: operators, making recursive calls only on smaller trees

# Merge Sort

Let's end with an example of a classic sorting algorithm.  First, assume we have two sorted lists.  How we can efficiently combine them together into one sorted list, where the occurrence count of each element equals the counts from the two input lists?  We can walk the two lists together from left to right, always picking the lowest unpicked element to add to an output list.

In [71]:
def merge(left, right):
    """Assumes left and right are sorted lists.  Returns a single new                    
    list built, in order, from the elements of left and right."""

    result = []
    i, j = 0, 0
    # Loop while there are elements in both lists.                                     
    while i < len(left) and j < len(right):
        # Copy smallest element to result.
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Copy over any remaining elements.
    # Only one of the lists has any elements remaining!
    result.extend(left[i:]) # Recall that extend adds all elements of the argument list.
    result.extend(right[j:])
    return result

In [123]:
assert(merge([1, 2], [3, 4])) == [1, 2, 3, 4]
assert(merge([1, 3], [2, 4])) == [1, 2, 3, 4]

Now the recipe helps us make a simple plan.

 1. *Divide the input space*: lists of length below two and at least two
 2. *Base cases*: length below two (the list is already sorted)
 3. *Recursive cases*: divide the input list roughly in half, so that each "half" is shorter than the original and thus legal to sort recursively

Merging is the perfect operation to combine results of recursive calls.

In [125]:
def sort(L):
    """Returns a new sorted list containing the same elements as L."""

    if len(L) < 2:
        return L[:] # Makes a fresh copy of L, just in case the caller mutates L later.
    else:
        # Split L into two lists of approximately equal lengths.  (They might be off by one.)
        middle = len(L)//2
        left = L[:middle]
        right = L[middle:]
        print('SPLIT into', left, 'and', right)
        
        # Sort the two sides.
        left = sort(left)
        right = sort(right)
        print('SORTED them to', left, 'and', right)
        
        # Merge the results.
        merged = merge(left, right)
        print('MERGED into', merged)
        return merged

In [126]:
assert sort([23, 3, 45, 7, 6, 11, 14, 12]) == [3, 6, 7, 11, 12, 14, 23, 45]

SPLIT into [23, 3, 45, 7] and [6, 11, 14, 12]
SPLIT into [23, 3] and [45, 7]
SPLIT into [23] and [3]
SORTED them to [23] and [3]
MERGED into [3, 23]
SPLIT into [45] and [7]
SORTED them to [45] and [7]
MERGED into [7, 45]
SORTED them to [3, 23] and [7, 45]
MERGED into [3, 7, 23, 45]
SPLIT into [6, 11] and [14, 12]
SPLIT into [6] and [11]
SORTED them to [6] and [11]
MERGED into [6, 11]
SPLIT into [14] and [12]
SORTED them to [14] and [12]
MERGED into [12, 14]
SORTED them to [6, 11] and [12, 14]
MERGED into [6, 11, 12, 14]
SORTED them to [3, 7, 23, 45] and [6, 11, 12, 14]
MERGED into [3, 6, 7, 11, 12, 14, 23, 45]
